In [1]:
import math
from scipy.optimize import minimize
import numpy as np

In [2]:
### tools：

def minfunction(f, x0, bounds):
    """
    求一个函数在边界内的最小值
    f：函数
    x0：初始值
    bounds：边界条件。[(min, max),...]
    """
    res = minimize(f, x0, method='SLSQP', bounds=bounds)
    return res

def maxfunction(f, x0, bounds):
    """
    求一个函数在边界内的最大值
    f：函数
    x0：初始值
    bounds：边界条件。[(min, max),...]
    """
    fun = lambda X : -f(X)
    res = minimize(fun, x0, method='SLSQP', bounds=bounds)
    return res


In [3]:
### 目标函数：

bounds = [(-2.3,2.3),(-2.3,2.3),(-3.2,3.2),(-3.2,3.2),(-3.2,3.2)]
X_optimal = [-1.71714224003, 1.59572124049468, 1.8272502406271, -0.763659881912867, -0.76365986736498]
y_optimal = 0.053941514041898

def G13(X):
    y = np.exp(X[0]*X[1]*X[2]*X[3]*X[4])
    h1 = X[0]**2+X[1]**2+X[2]**2+X[3]**2+X[4]**2-10-0.0001
    h2 = X[1]*X[2]-5*X[3]*X[4]-0.0001
    h22 = -X[1]*X[2]+5*X[3]*X[4]-0.0001
    h3 = X[0]*X[0]*X[0]+X[1]*X[1]*X[1]+1-0.0001
    h33 = -X[0]*X[0]*X[0]-X[1]*X[1]*X[1]-1-0.0001
    return y,h1,h2,h22,h3,h33

In [4]:
np.e**(-2.3*2.3*3.2*3.2*3.2)

5.2265273323504526e-76

In [5]:
# y = lambda X: G13(X)[0]
# h1 = lambda X: G13(X)[1]
# h2 = lambda X: G13(X)[2]
# h3 = lambda X: G13(X)[3]
# x0 = [0,0,0,0,0]
# x0 = [-2.3,-2.3,-2.3,-2.3,-2.3]

In [6]:
# maxfunction(h3,x0,bounds)

In [7]:
### 适应度函数
def fitnessG13_num(X, epsion):
    """
    problem1关于数量惩罚的适应度函数
    X: 对应的解
    epsion：惩罚系数大于fmax-fmin的倍数
    """
    fmax = 1220.174
    fmin = 0.008509619375000003
    count = np.sum(np.array(problem1(X))[1:5]>0)
    
    return -(problem1(X)[0]+epsion*(fmax-fmin)*count)  

def fitnessG13_vio(X):
    """
    problem1关于数量惩罚的适应度函数
    X: 对应的解
    epsion：惩罚系数大于fmax-fmin的倍数
    """
    fmax = 1.913316311980873e+75
    fmin = 5.2265273323504526e-76
    vmax = 64
    e = 0.001
    n = 5
    
    res = G13(X)
    fitness = -res[0]
    count = np.sum(np.array(res)[1:6]>0)
    
    
    if count > 0:
        for i in range(1,len(res)):
            if res[i] > 0:
                cathy = (fmax - fmin)*(e*vmax*n/count+res[i])/(e*vmax)
                fitness = fitness - cathy
    return fitness   

In [8]:
### PSO
class PSO_fit():
    def __init__(self, pN, dim, max_iter, bounds):
        self.wmax = 0.8 #最大的惯性权重
        self.wmin = 0.4 #最小的惯性权重
        self.c1 = 2  #加速因子
        self.c2 = 2  #加速因子
        self.pN = pN  # 粒子数量
        self.dim = dim  # 搜索维度
        self.max_iter = max_iter  # 迭代次数
        self.bounds = bounds
        
        self.X = np.zeros((self.pN, self.dim))  # 所有粒子的位置：一个pN*dim的矩阵
        self.V = np.zeros((self.pN, self.dim))  # 左右粒子的速度：一个pN*dim的矩阵
        self.pbest = np.zeros((self.pN, self.dim))  # 个体经历的最佳位置：一个pN*dim的矩阵
        self.gbest = np.zeros((self.pN, self.dim))      # 全局最佳位置：一个pN*dim的矩阵
        self.p_fit = np.full(self.pN,-10.0e+77)  # 每个个体的历史最佳适应值
        self.g_fit = np.full(self.pN,-10.0e+77)    # 全局最佳适应值
    
    def init_Population(self):
        """
        初始化种群
        :return:
        """
        
        for i in range(self.pN):
            for j in range(self.dim):
                self.X[i][j] = self.bounds[j][0] + (self.bounds[j][1] - self.bounds[j][0]) * np.random.random()
                self.V[i][j] = self.bounds[j][0] + (self.bounds[j][1] - self.bounds[j][0]) * np.random.random()
        for i in range(self.pN):
            tmp = fitnessG13_vio(self.X[i])
            if (tmp > self.p_fit[i]):
                self.p_fit[i] = tmp
                self.pbest[i] = self.X[i]
                
        for i in range(self.pN):
            indexes = np.random.randint(low=0,high=self.pN,size=int(self.pN/400))
            for j in indexes:
                if self.p_fit[j] > self.g_fit[i]:
                    self.g_fit[i] = self.p_fit[j]
                    self.gbest[i] = self.pbest[j]

#             self.pbest[i] = self.X[i]
#             tmp = fitnessG01_num(self.X[i])
#             self.p_fit[i] = tmp
#             if (tmp > self.fit):
#                 self.fit = tmp
#                 self.gbest = self.X[i]
        
    def iterator(self):
        """
        迭代
        """
        for t in range(self.max_iter):
#             w = self.wmin + (self.wmax - self.wmin) * np.exp(- (4 * t / self.max_iter) * (4 * t / self.max_iter))
            w = 0.5
            for i in range(self.pN):
                self.V[i] = w*self.V[i]+self.c1*np.random.random()*(self.pbest[i]-self.X[i])+self.c2*np.random.random()*(self.gbest[i]-self.X[i])
                self.X[i] = self.X[i] + self.V[i]
                
                """
                控制在搜索范围内
                """ 
                for j in range(self.dim):
                    if self.X[i][j] < self.bounds[j][0]:
                        self.X[i][j] = self.bounds[j][0]
                    if self.X[i][j] > self.bounds[j][1]:
                        self.X[i][j] = self.bounds[j][1]     
            for i in range(self.pN):
                tmp = fitnessG13_vio(self.X[i])
                if (tmp > self.p_fit[i]):
                    self.p_fit[i] = tmp
                    self.pbest[i] = self.X[i]
            for i in range(self.pN):
                indexes = np.random.randint(low=0,high=self.pN,size=int(self.pN/400))
                for j in indexes:
                    if self.p_fit[j] > self.g_fit[i]:
                        self.g_fit[i] = self.p_fit[j]
                        self.gbest[i] = self.pbest[j]
            print(self.g_fit)  
        return self.gbest, self.g_fit

In [9]:
PSO = PSO_fit(pN=8000, dim=5, max_iter=10000, bounds=bounds)

In [10]:
PSO.init_Population()

In [ ]:
PSO.iterator()

[-1.09012457e+77 -1.10980200e+77 -5.09813665e+76 ... -9.18559747e+76
 -6.26283400e+76 -5.49425894e+76]
[-2.97108880e+76 -5.30660036e+76 -2.59910098e+76 ... -2.35442628e+76
 -4.35237699e+76 -5.35662980e+76]
[-1.59612091e+76 -2.50877471e+76 -2.29794581e+76 ... -1.35466256e+76
 -4.34283919e+76 -2.53514670e+76]
[-1.59612091e+76 -2.50877471e+76 -1.81866796e+76 ... -1.35466256e+76
 -2.61083246e+76 -2.53514670e+76]
[-1.59612091e+76 -1.38209649e+76 -1.81866796e+76 ... -1.35466256e+76
 -1.23701745e+76 -2.53514670e+76]
[-1.59612091e+76 -1.38209649e+76 -1.81866796e+76 ... -1.35466256e+76
 -1.23701745e+76 -2.53514670e+76]
[-1.20232625e+76 -1.38209649e+76 -1.65677919e+76 ... -1.35466256e+76
 -1.23701745e+76 -2.05574584e+76]
[-1.05011086e+76 -1.38209649e+76 -1.65509725e+76 ... -1.35466256e+76
 -1.23701745e+76 -1.50760721e+76]
[-1.05011086e+76 -1.38209649e+76 -1.30826589e+76 ... -1.35033891e+76
 -1.03140913e+76 -1.36369364e+76]
[-1.05011086e+76 -1.38209649e+76 -1.30826589e+76 ... -1.35033891e+76
 -1.

[-0.99927784 -0.9992773  -0.99930902 ... -0.99927311 -0.99928681
 -0.99955779]
[-0.99927784 -0.9992773  -0.99930902 ... -0.99927311 -0.99928507
 -0.99955779]
[-0.99927784 -0.9992773  -0.99930902 ... -0.99925024 -0.99928507
 -0.99955779]
[-0.99927784 -0.9992773  -0.99930902 ... -0.99925024 -0.99928507
 -0.99955779]
[-0.99927784 -0.99927098 -0.99930902 ... -0.99925024 -0.99928507
 -0.99955779]
[-0.99927784 -0.99927098 -0.99926215 ... -0.99925024 -0.99928507
 -0.99955779]
[-0.99927784 -0.99927098 -0.99926215 ... -0.99925024 -0.99918938
 -0.99924171]
[-0.99927784 -0.99927098 -0.99926215 ... -0.99925024 -0.99918938
 -0.99924171]
[-0.99927784 -0.99927098 -0.99926215 ... -0.99916786 -0.99918938
 -0.99924171]
[-0.99927784 -0.99917218 -0.99893799 ... -0.99916786 -0.99918938
 -0.99924171]
[-0.99927784 -0.99917218 -0.99893799 ... -0.99915201 -0.99918938
 -0.99924171]
[-0.99927784 -0.99917218 -0.99893799 ... -0.99915201 -0.99918938
 -0.99924171]
[-0.99927784 -0.99917218 -0.99893799 ... -0.99915201

[-0.89125098 -0.88733465 -0.88583053 ... -0.88925648 -0.88456761
 -0.88800925]
[-0.88149894 -0.88733465 -0.88583053 ... -0.88925648 -0.88405819
 -0.88800925]
[-0.88149894 -0.88733465 -0.88505323 ... -0.88322641 -0.88405819
 -0.8841713 ]
[-0.88149894 -0.88733465 -0.88505323 ... -0.88322641 -0.88405819
 -0.8841713 ]
[-0.88149894 -0.88733465 -0.88505323 ... -0.88322641 -0.88087216
 -0.88112648]
[-0.88149894 -0.88733465 -0.88230896 ... -0.88322641 -0.88087216
 -0.87824651]
[-0.8791181  -0.88598949 -0.88230896 ... -0.88243268 -0.88087216
 -0.87824651]
[-0.8791181  -0.88108208 -0.88230896 ... -0.88243268 -0.88087216
 -0.87824651]
[-0.8791181  -0.88108208 -0.88230896 ... -0.88007798 -0.88087216
 -0.87824651]
[-0.8769686  -0.88093284 -0.88168492 ... -0.88007798 -0.88020742
 -0.87666097]
[-0.8769686  -0.88093284 -0.87949986 ... -0.87966599 -0.87745358
 -0.87666097]
[-0.8769686  -0.88019813 -0.87949986 ... -0.87709192 -0.87745358
 -0.87666097]
[-0.87509412 -0.88019813 -0.87414688 ... -0.87709192

[-0.79749948 -0.79944785 -0.80262356 ... -0.80292027 -0.80178123
 -0.80232638]
[-0.79749948 -0.79944785 -0.80249382 ... -0.80013205 -0.80148109
 -0.80050911]
[-0.79749948 -0.79944785 -0.80101089 ... -0.80013205 -0.80148109
 -0.80050911]
[-0.79749948 -0.79944785 -0.80101089 ... -0.80013205 -0.79867451
 -0.80050911]
[-0.79749948 -0.79944785 -0.80101089 ... -0.80013205 -0.79867451
 -0.79985783]
[-0.79610724 -0.79867451 -0.79737945 ... -0.80013205 -0.79782747
 -0.7968991 ]
[-0.79610724 -0.79867451 -0.79737945 ... -0.80013205 -0.79782747
 -0.7968991 ]
[-0.79610724 -0.79722061 -0.79710539 ... -0.79910392 -0.79782747
 -0.7968991 ]
[-0.79610724 -0.79722061 -0.79710539 ... -0.79910392 -0.79782747
 -0.7960022 ]
[-0.79507417 -0.79722061 -0.79306316 ... -0.79910392 -0.79347578
 -0.7960022 ]
[-0.79507417 -0.79636986 -0.79306316 ... -0.78814812 -0.79347578
 -0.7960022 ]
[-0.79507417 -0.79636986 -0.79306316 ... -0.78814812 -0.79347578
 -0.7960022 ]
[-0.79346803 -0.79636986 -0.79306316 ... -0.78814812

[-0.70917415 -0.71212591 -0.71313512 ... -0.71328161 -0.7091489
 -0.71407318]
[-0.70901349 -0.71212591 -0.71313512 ... -0.70846301 -0.7091489
 -0.71407318]
[-0.70901349 -0.71212591 -0.70624946 ... -0.70846301 -0.7091489
 -0.71407318]
[-0.70782395 -0.71212591 -0.70624946 ... -0.70846301 -0.70892977
 -0.70988692]
[-0.70782395 -0.71035406 -0.70624946 ... -0.70846301 -0.70892977
 -0.70988692]
[-0.70782395 -0.70772845 -0.70624946 ... -0.70846301 -0.70892977
 -0.70988692]
[-0.70782395 -0.70772845 -0.70624946 ... -0.70612203 -0.70892977
 -0.70914585]
[-0.70782395 -0.70772845 -0.70113731 ... -0.70612203 -0.70298154
 -0.70678331]
[-0.6983221  -0.70742245 -0.70113731 ... -0.70448539 -0.70298154
 -0.70327709]
[-0.6983221  -0.70742245 -0.70113731 ... -0.70049679 -0.70298154
 -0.70327709]
[-0.6983221  -0.70742245 -0.70113731 ... -0.70049679 -0.70219941
 -0.69767591]
[-0.6983221  -0.70507496 -0.70113731 ... -0.70049679 -0.70219941
 -0.69767591]
[-0.6983221  -0.69977348 -0.70113731 ... -0.70049679 -0

...

[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151

[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151

[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151

[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151

[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151 -0.05394151
 -0.05394151]
[-0.05394151 -0.05394151 -0.05394151 ... -0.05394151

In [ ]:
PSO.gbest[0]

In [ ]:
G13(PSO.gbest[0])